# Machine Learning 

In [11]:
import pandas as pd
import numpy as np
import os
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt  
% matplotlib inline
import random
from datetime import datetime
import seaborn as sns
from pandas.tools.plotting import scatter_matrix
from scipy.sparse import coo_matrix
import copy
import sklearn.neighbors, sklearn.linear_model, sklearn.ensemble, sklearn.naive_bayes # Baseline classification techniques
import scipy.io # Import data
import time

In [29]:
df_train_users = pd.read_csv("train_users2_cleaned.csv")
#df_sessions = pd.read_csv("session_cleaned.csv")
df_test_users = pd.read_csv("test_users_cleaned.csv")
df_train_users.head()

,Unnamed: 0,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
1,3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
2,4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,UNKNOWN,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
3,6,lsw9q7uk0j,2010-01-02,20100102012558,2010-01-05,FEMALE,46.0,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US
4,7,0d01nltbrs,2010-01-03,20100103191905,2010-01-13,FEMALE,47.0,basic,0,en,direct,direct,omg,Web,Mac Desktop,Safari,US


## 1. From data frame to matrix : Construct y_train

What we want now for the training is 2 matrices X_train (matrix of relevant features) and y_train (booking dest)

In [13]:
# extract y_train
y_train = df_train_users['country_destination']

# gen var
country_dest = df_train_users['country_destination'].unique()
nb_dest = len(country_dest)
y_train_len = len(y_train)

# mapping 
country_to_ix = { co:i for i,co in enumerate(country_dest) }
ix_to_country = { i:co for i,co in enumerate(country_dest) }

y_train_ix = [country_to_ix[co] for co in y_train[:y_train_len]]
y_train = np.array(y_train_ix)

print('\nMapping country to numbers:',country_to_ix)
print('\nMapping numbers to country:',ix_to_country)


Mapping country to numbers: {'AU': 9, 'US': 0, 'IT': 4, 'FR': 3, 'NL': 7, 'GB': 6, 'other': 1, 'DE': 8, 'CA': 2, 'PT': 10, 'ES': 5}

Mapping numbers to country: {0: 'US', 1: 'other', 2: 'CA', 3: 'FR', 4: 'IT', 5: 'ES', 6: 'GB', 7: 'NL', 8: 'DE', 9: 'AU', 10: 'PT'}


In [14]:
# Convert label values to one_hot vector 
one_hot_enable = 0
def convert_to_one_hot(a,max_val=None):
    N = a.size
    data = np.ones(N,dtype=int)
    sparse_out = coo_matrix((data,(np.arange(N),a.ravel())), shape=(N,max_val))
    return np.array(sparse_out.todense())

if one_hot_enable:
    y_train = convert_to_one_hot(y_train,nb_dest)

print(np.size(y_train))
print(y_train)

67163
[0 1 0 ..., 5 0 0]


## 2. From data frame to matrix : Construct X_train

From the cleaned files sessions and train_users we need to choose (relevant) features. 
Ex :
    - number of sessions per id 
    - type of actions 
    - number of actions (number of clicks in total)
    - device 
    - average time per session 
    - etc...

In [30]:
# Drop id, country_destination & first_date_booking
df_train_users = df_train_users.drop(['id','date_first_booking','country_destination'])

#One-hot-encoding features
feats = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider',
             'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']
for f in feats:
    df_all_dummy = pd.get_dummies(df_train_users[f], prefix=f)
    df_train_users = df_train_users.drop([f], axis=1)
    df_train_users = pd.concat((df_train_users, df_all_dummy), axis=1)
    

In [32]:
print(df_train_users.shape)
print(df_train_users)

(67163, 132)
       Unnamed: 0          id date_account_created  timestamp_first_active  \
0               2  4ft3gnwmtx           2010-09-28          20090609231247   
1               3  bjjt8pjhuk           2011-12-05          20091031060129   
2               4  87mebub9p4           2010-09-14          20091208061105   
3               6  lsw9q7uk0j           2010-01-02          20100102012558   
4               7  0d01nltbrs           2010-01-03          20100103191905   
5               8  a1vcnhxeij           2010-01-04          20100104004211   
6               9  6uh8zyj2gn           2010-01-04          20100104023758   
7              10  yuuqmid2rp           2010-01-04          20100104194251   
8              15  v4d5rl22px           2010-01-07          20100107204555   
9              17  frhre329au           2010-01-07          20100107224625   
10             19  gdka1q5ktd           2010-01-10          20100110010817   
11             21  qsibmuz9sx           2010-01-10 

## Run machine learning with X_train and y_train

In [18]:
# Train model example with random forest 
#rand_forest_model = sklearn.ensemble.RandomForestClassifier(n_estimators=100,max_depth=15)
#rand_forest_model.fit(X_train,y_train)

# Predict from model neigh
#y_pred = rand_forest_model.predict(X_test)

# random country for testing 
y_test_len = len(df_test_users)
y = np.random.randint(nb_dest, size=y_test_len)
# ix to country 
y_pred = []
for ix in list(y):
    #y_pred.append(ix_to_country[ix])
    y_pred.append("NDF")
 

## Convert to csv for submission

In [22]:
df_submission = pd.DataFrame(np.column_stack((df_test_users['id'], df_y_pred)), columns=['id', 'country'])
df_submission.to_csv('submission_country_dest.csv',index=False)